In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

## Obtain Data
The stock price data will be obtained through Yahoo Finance throughout the course.

In [ ]:
import yfinance as yf

# Get the data for the stock AAPL
ford = yf.download('F','2018-01-01', datetime.today(), progress=False)['Adj Close']
ge = yf.download('GE','2016-01-01', datetime.today(), progress=False)['Adj Close']

Quick visualizations

## Merging series
**Exercise 1**: Consolidate the Data of Ford and GE into one single DataFrame based on the date index.

**Exercise 2**: Remove all the entries where there are missing values

**Exercise**: What values do you have for Jan 8th, 2021 ?

## Financial Analysis
**Exercise**: calculate the difference between the return of GE and Ford

**Exercise**: obtain the log returns for both assets (log(1+x))

**Exercise**: compute the rolling window volatility (annualized) using a window size of 20

In [ ]:
fig , ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(df_returns.index, df_returns['Ford_volatility']*100,
        label='$\sigma$ (F): '+ str(100*np.round(df_returns['Ford_volatility'].mean(),3))+ ' %')
ax.plot(df_returns.index, df_returns['GE_volatility']*100,
        label='$\sigma$ (F): '+ str(100*np.round(df_returns['GE_volatility'].mean(),3))+ ' %')

ax.set_title('30 Day Vol',fontsize=14)
ax.legend(fontsize=14)
ax.set_ylabel('Volatility %');

**Exercise**: compute the rolling correlation (window size=100)

In [ ]:
df_returns.dropna(how='any',inplace=True)